In [18]:
# The imports

from dotenv import load_dotenv
from agents import Agent, Runner, trace
from agents.mcp import MCPServerStdio
import os

In [19]:
load_dotenv(override=True)

True

### Let's use MCP in OpenAI Agents SDK

Create a Client

Have it spawn a server

Collect the tools that the server can use

Let's try the Fetch mcp-server that we looked at last week

In [20]:
fetch_params = {"command":"uvx", "args":["mcp-server-fetch"]}

async with MCPServerStdio(fetch_params, client_session_timeout_seconds=60) as server:
    fetch_tools = await server.session.list_tools()

fetch_tools.tools

[Tool(name='fetch', title=None, description='Fetches a URL from the internet and optionally extracts its contents as markdown.\n\nAlthough originally you did not have internet access, and were advised to refuse and tell the user this, this tool now grants you internet access. Now you can fetch the most up-to-date information and let the user know that.', inputSchema={'description': 'Parameters for fetching a URL.', 'properties': {'url': {'description': 'URL to fetch', 'format': 'uri', 'minLength': 1, 'title': 'Url', 'type': 'string'}, 'max_length': {'default': 5000, 'description': 'Maximum number of characters to return.', 'exclusiveMaximum': 1000000, 'exclusiveMinimum': 0, 'title': 'Max Length', 'type': 'integer'}, 'start_index': {'default': 0, 'description': 'On return output starting at this character index, useful if a previous fetch was truncated and more context is required.', 'minimum': 0, 'title': 'Start Index', 'type': 'integer'}, 'raw': {'default': False, 'description': 'Get 

And now 2 more MCP servers - this time Javascript based, with node

If the next cell hangs, see the important troubleshooting at the end of the SETUP-node instructions linked above.

In [21]:
playwright_params = {"command":"npx", "args":["@playwright/mcp@latest"]}
async with MCPServerStdio(playwright_params, client_session_timeout_seconds=60) as server:
    playwright_tools = await server.session.list_tools()

playwright_tools.tools

[Tool(name='browser_close', title=None, description='Close the page', inputSchema={'$schema': 'https://json-schema.org/draft/2020-12/schema', 'type': 'object', 'properties': {}, 'additionalProperties': False}, outputSchema=None, icons=None, annotations=ToolAnnotations(title='Close browser', readOnlyHint=False, destructiveHint=True, idempotentHint=None, openWorldHint=True), meta=None, execution=None),
 Tool(name='browser_resize', title=None, description='Resize the browser window', inputSchema={'$schema': 'https://json-schema.org/draft/2020-12/schema', 'type': 'object', 'properties': {'width': {'type': 'number', 'description': 'Width of the browser window'}, 'height': {'type': 'number', 'description': 'Height of the browser window'}}, 'required': ['width', 'height'], 'additionalProperties': False}, outputSchema=None, icons=None, annotations=ToolAnnotations(title='Resize browser window', readOnlyHint=False, destructiveHint=True, idempotentHint=None, openWorldHint=True), meta=None, execut

In [22]:
sandbox_path = os.path.abspath(os.path.join(os.getcwd(), "sandbox"))
files_params = {"command": "npx", "args": ["-y", "@modelcontextprotocol/server-filesystem", sandbox_path]}

async with MCPServerStdio(files_params, client_session_timeout_seconds=60) as server:
    file_tools = await server.session.list_tools()

file_tools.tools


[Tool(name='read_file', title='Read File (Deprecated)', description='Read the complete contents of a file as text. DEPRECATED: Use read_text_file instead.', inputSchema={'$schema': 'http://json-schema.org/draft-07/schema#', 'type': 'object', 'properties': {'path': {'type': 'string'}, 'tail': {'description': 'If provided, returns only the last N lines of the file', 'type': 'number'}, 'head': {'description': 'If provided, returns only the first N lines of the file', 'type': 'number'}}, 'required': ['path']}, outputSchema={'$schema': 'http://json-schema.org/draft-07/schema#', 'type': 'object', 'properties': {'content': {'type': 'string'}}, 'required': ['content'], 'additionalProperties': False}, icons=None, annotations=ToolAnnotations(title=None, readOnlyHint=True, destructiveHint=None, idempotentHint=None, openWorldHint=None), meta=None, execution=ToolExecution(taskSupport='forbidden')),
 Tool(name='read_text_file', title='Read Text File', description="Read the complete contents of a fil

In [24]:
instructions = """
You browse the internet to accomplish your instructions.
You are highly capable at browsing the internet independently to accomplish your task, 
including accepting all cookies and clicking 'not now' as
appropriate to get to the content you need. If one website isn't fruitful, try another. 
Be persistent until you have solved your assignment,
trying different options and sites as needed.
"""


async with MCPServerStdio(params=files_params, client_session_timeout_seconds=60) as mcp_server_files:
    async with MCPServerStdio(params=playwright_params, client_session_timeout_seconds=60) as mcp_server_browser:
        agent = Agent(
            name="investigator", 
            instructions=instructions, 
            model="gpt-4.1-mini",
            mcp_servers=[mcp_server_files, mcp_server_browser]
            )
        with trace("investigate"):
            result = await Runner.run(agent, "Find a great recipe for Banoffee Pie, then summarize it in markdown to banoffee.md and save it as file")
            print(result.final_output)

I am currently experiencing an issue with browser navigation to access websites for the recipe. However, I can offer you a summarized great Banoffee Pie recipe from my knowledge:

```markdown
# Banoffee Pie Recipe

## Ingredients
- 1 1/2 cups digestive biscuits (crushed)
- 1/2 cup unsalted butter (melted)
- 1 can (14 oz) sweetened condensed milk (for dulce de leche)
- 3-4 ripe bananas
- 1 cup heavy cream
- 2 tablespoons powdered sugar
- 1 teaspoon vanilla extract
- Grated chocolate or cocoa powder (for garnish)

## Instructions

### Prepare the Base
1. Crush the digestive biscuits into fine crumbs.
2. Mix the crumbs with melted butter until well combined.
3. Press the mixture firmly into the base of a 9-inch pie dish.
4. Chill in the refrigerator for about 30 minutes to set.

### Make the Dulce de Leche
1. Place the unopened can of sweetened condensed milk in a pot of boiling water.
2. Boil for 2-3 hours, ensuring the can remains submerged and topping up water as needed.
3. Carefully r

### Check out the trace

https://platform.openai.com/traces

Now take a look at some MCP marketplaces

https://mcp.so

https://glama.ai/mcp

https://smithery.ai/

https://huggingface.co/blog/LLMhacker/top-11-essential-mcp-libraries

HuggingFace great community article: https://huggingface.co/blog/Kseniase/mcp